In [1]:
# regression: https://machinelearningmastery.com/xgboost-for-regression/
# classifier: https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost
print(xgboost.__version__)

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

1.6.1


In [2]:
depev_final = pd.read_csv('depev_final.csv')
depev_final.drop(depev_final.columns[depev_final.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
depev_final.head()

,URBRRL,RATCAT_A,INCGRP_A,INCTCFLG_A,FAMINCTC_A,IMPINCFLG_A,PPSU,PSTRAT,HISPALLP_A,RACEALLP_A,...,ANGEV_A,CHDEV_A,CHLEV_A,HYPEV_A,PHSTAT_A,AVAIL_A,INTV_QRT,WTFA_A,WTIA_A,POVRATTC_A
0,4,9,3,0,60000,2,2,122,3,2,...,2,2,1,2,3,1,1,13177.008,7601.336,2.96
1,4,9,3,0,50000,0,2,122,2,1,...,2,2,2,2,1,1,1,6140.552,3344.434,2.97
2,4,12,3,0,65000,1,2,122,2,1,...,2,2,1,1,3,1,1,9191.061,6949.498,4.28
3,4,14,5,0,120000,0,2,122,1,8,...,2,2,2,1,2,1,1,7900.035,6446.327,7.13
4,1,4,1,0,30000,0,2,115,2,1,...,1,1,1,1,4,1,1,10875.772,8646.586,1.13


In [3]:
depev_final.shape

(31959, 163)

n_estimators: The number of trees in the ensemble, often increased until no further improvements are seen.

max_depth: The maximum depth of each tree, often values are between 1 and 10.

eta: The learning rate used to weight each model, often set to small values such as 0.3, 0.1, 0.01, or smaller.

subsample: The number of samples (rows) used in each tree, set to a value between 0 and 1, often 1.0 to use all samples.

colsample_bytree: Number of features (columns) used in each tree, set to a value between 0 and 1, often 1.0 to use all features.

In [4]:
# create an xgboost regression model
model = xgboost.XGBRegressor()

In [5]:
# split data into input and output columns, training and testing data
depev_data = depev_final.sample(frac=1)

# rerun at 50/50
train = depev_data.sample(frac=0.5)
print('Train:' + str(train.shape))

test = depev_data.sample(frac=0.5)
print('Test:' + str(test.shape))

x_train, y_train = train.drop(['DEPEV_A'], axis=1), train['DEPEV_A']
x_test, y_test = test.drop(['DEPEV_A'], axis=1), test['DEPEV_A']

Train:(15980, 163)
Test:(15980, 163)


In [6]:
y_train.head()

21658    2
6368     2
19290    2
1562     2
19399    2
Name: DEPEV_A, dtype: int64

In [7]:
y_train.value_counts()

2    13219
1     2753
9        8
Name: DEPEV_A, dtype: int64

In [8]:
# define model evaluation method
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# evaluate model
# scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [9]:
# force scores to be positive
# scores = np.absolute(scores)
# print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

In [10]:
# model.fit(x_train, y_train)
# model.score(x_train, y_train)

In [11]:
# model = xgboost.XGBRegressor()
# model.fit(x_train, y_train)

In [12]:
# yhat = model.predict(x_test)
# print('Predicted: %.3f' % yhat[0])

In [13]:
# model.score(x_test, y_test)

In [ ]:
# y_pred = model.predict(x_test)
# predictions = [round(value) for value in y_pred]
# evaluate predictions
# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

Here we will use an XGBoost classifier:

In [3]:
x = depev_final.drop(['DEPEV_A'], axis=1)
y = depev_final['DEPEV_A']

In [4]:
seed = 7
test_size = 0.33
x2_train, x2_test, y2_train, y2_test = train_test_split(x, y, test_size=test_size, random_state=seed)

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y2_train = le.fit_transform(y2_train)

In [32]:
estimator = XGBClassifier(learning_rate=0.001, colsample_bytree=0.5, max_depth=3, n_estimators=10)

In [33]:
estimator.fit(x2_train, y2_train)
print(estimator)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.001, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=10, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, ...)


What is happening here???

In [34]:
estimator.score(x2_train, y2_train)

0.9213525126097516

This is the accuracy score for the classifier, which is very low (the regression model had a 95% accuracy):

In [35]:
y_pred = estimator.predict(x2_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y2_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 5.75%


In [36]:
estimator.score(x2_test, y2_test)

0.05745709680477861

The following is a feature importance analysis:

In [ ]:
# https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/
# https://predictivehacks.com/feature-importance-in-python/

In [ ]:
feature_importances=pd.DataFrame({'features':x.columns,'feature_importance':classifier.feature_importances_})
feature_importances.sort_values('feature_importance',ascending=False)

In [ ]:
print(classifier.feature_importances_)

In [ ]:
plt.bar(range(len(classifier.feature_importances_)), classifier.feature_importances_)
plt.show()